#### **Attention is All You Need (NIPS 2017)** 실습
https://github.com/ndb796/Deep-Learning-Paper-Review-and-Practice

#### <b>BLEU Score 계산을 위한 라이브러리 업데이트</b>
* 이를 사용하기 위해 torchtext==0.6.0 버전으로 설치
* <b>[Restart Runtime]</b> 버튼을 눌러 런타임을 재시작할 필요가 있습니다.

In [119]:
# !pip install torchtext==0.6.0

In [120]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random

SEED = 777

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

#### **데이터 전처리(Preprocessing)**

* **spaCy 라이브러리**: 문장의 토큰화(tokenization), 태깅(tagging) 등의 전처리 기능을 위한 라이브러리
  * 영어(Engilsh)와 독일어(Deutsch) 전처리 모듈 설치

In [121]:
# %%capture
#   # 출력을 바로 아래에 표시되지 않고 %%capture captured_output 면
#   # captured_output을 접근하여 캡처된 출력을 프로그래밍적으로 확인하거나 처리
# !python -m spacy download en
# !python -m spacy download de

In [122]:
import spacy

spacy_en = spacy.load('en_core_web_sm')

In [123]:
# 간단히 토큰화(tokenization) 기능 써보기
spacy_en.tokenizer("I am a graduate student.")

I am a graduate student.

* 영어(English) 및 독일어(Deutsch) **토큰화 함수** 정의

In [124]:
from transformers import CamembertTokenizer, CamembertModel

bert_model_name = 'camembert-base'
# bert_model_name = 'camembert/camembert-large'
tokenizer = CamembertTokenizer.from_pretrained(bert_model_name) # local_files_only=True)
bert_model = CamembertModel.from_pretrained(bert_model_name)

# camembert.eval()  # disable dropout (or leave in train mode to finetune)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

bert_model = bert_model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


cuda


In [125]:
### camembert-base 크게 세 가지
# bert_model.embeddings 
# bert_model.encoder
# bert_model.pooler
# bert_model

In [126]:
# for param in bert_model.parameters():
# for param in bert_model.embeddings.parameters():
for name, param in bert_model.named_parameters():
    # print(name)
    if 'encoder.layer.11.output' not in name and 'pooler' not in name:
        param.requires_grad = False

In [127]:
for name, param in bert_model.named_parameters():
    # <는 왼쪽 정렬, 숫자는 해당 필드의 최소 폭
    print("{:<50} {}".format(name, param.requires_grad))

embeddings.word_embeddings.weight                  False
embeddings.position_embeddings.weight              False
embeddings.token_type_embeddings.weight            False
embeddings.LayerNorm.weight                        False
embeddings.LayerNorm.bias                          False
encoder.layer.0.attention.self.query.weight        False
encoder.layer.0.attention.self.query.bias          False
encoder.layer.0.attention.self.key.weight          False
encoder.layer.0.attention.self.key.bias            False
encoder.layer.0.attention.self.value.weight        False
encoder.layer.0.attention.self.value.bias          False
encoder.layer.0.attention.output.dense.weight      False
encoder.layer.0.attention.output.dense.bias        False
encoder.layer.0.attention.output.LayerNorm.weight  False
encoder.layer.0.attention.output.LayerNorm.bias    False
encoder.layer.0.intermediate.dense.weight          False
encoder.layer.0.intermediate.dense.bias            False
encoder.layer.0.output.dense.we

In [128]:
# 독일어(Deutsch) 문장을 토큰화 하는 함수 (순서를 뒤집지 않음)
def tokenize_de(text):
    return tokenizer.encode(text, add_special_tokens=False)

# 영어(English) 문장을 토큰화 하는 함수
def tokenize_en(text):
    return [token.text for token in spacy_en.tokenizer(text)]

* **필드(field)** 라이브러리를 이용해 데이터셋에 대한 구체적인 전처리 내용을 명시합니다.
    * init_token="<sos>" : 번역 모델의 입력으로 넣을 때 각각의 문장들의 앞부분에는 sos 토큰을 붙임
    * lower : 각 단어를 소문자로 바꾸는 것이 일반적
    * batch_first : 트랜스포머의 입력을 넣을 때는 텐서의 차원에서 시퀀스 보다 배치가 먼저 오도록
* Seq2Seq 모델과는 다르게 <b>batch_first 속성의 값을 True로 설정</b>합니다.
* 번역 목표
    * 소스(SRC): 독일어
    * 목표(TRG): 영어

In [129]:
from torchtext.data import Field, BucketIterator

SRC = Field(
    use_vocab=False,
    tokenize=tokenize_de,
    pad_token=tokenizer.pad_token_id,  # 패딩 토큰 ID 설정 @단어 집합의 문자열 부분도 숫자로 들어감 
    init_token=tokenizer.cls_token_id,  # 시퀀스 시작 토큰 ID 설정
    eos_token=tokenizer.sep_token_id,  # 문장 종료 토큰 ID 설정
    unk_token=tokenizer.unk_token_id,  # 알 수 없는 토큰 ID 설정
    batch_first=True
)
TRG = Field(tokenize=tokenize_en, init_token="<sos>", eos_token="<eos>", lower=True, batch_first=True)

* 대표적인 영어-독어 번역 데이터셋인 **Multi30k**를 불러옵니다.
    * 약 3만개의 영어 독일어 쌍을 가짐
    * fields=(SRC, TRG) : 위 Field 라이버르러리를 이용해서 독일어를 영어로 바꾸는 태스크에 대해서 각각 앞서 정의했던 전처리(SRC, TRG)를 수행할 수 있도록

In [130]:
filepath = '/data/hwyu/data/libri/'

train_en_path = filepath+"train/train.en"
train_de_path = filepath+"train/train_gtranslate.fr" 

train_oth_en_path = filepath+"train/other.en"
train_oth_de_path = filepath+"train/other_gtranslate.fr" 

test_eng_path = filepath+"test/test.en"
test_de_path = filepath+"test/test_gtranslate.fr"

test_dev_eng_path = filepath+"test/dev.en"
test_dev_de_path = filepath+"test/dev_gtranslate.fr"

In [131]:
end_line = 100000  # 끝 라인 (포함하지 않음)

train_en_sub_raw = []  # 추출한 데이터를 저장할 리스트

with open(train_en_path, "r", encoding="utf-8") as file:
    for i, line in enumerate(file):
        train_en_sub_raw.append(line.strip())
        if i + 1 >= end_line:
            break

with open(train_oth_en_path, "r", encoding="utf-8") as file:
    for i, line in enumerate(file):
        train_en_sub_raw.append(line.strip())
        if i + 1 >= end_line:
            break
            
limit_data_len = 120000
train_en_sub_raw = train_en_sub_raw[:limit_data_len]
# 데이터 확인
print(len(train_en_sub_raw))
print(train_en_sub_raw[:10])

108640
['ADIEU VALENTINE ADIEU', 'PROVE IT DANGLARS', 'SAID FRANZ', 'SAID FRANZ', 'FERNAND MONDEGO', 'AND THE CORRIDOR', 'FOR ANDREA RONDOLO', 'FOR ANDREA RONDOLO', 'SAID FERNAND', 'SAID ANDREA']


In [132]:
train_fr_sub_raw = []

with open(train_de_path, "r", encoding="utf-8") as file:
    for i, line in enumerate(file):
        train_fr_sub_raw.append(line.strip())
        if i + 1 >= end_line:
            break

with open(train_oth_de_path, "r", encoding="utf-8") as file:
    for i, line in enumerate(file):
        train_fr_sub_raw.append(line.strip())
        if i + 1 >= end_line:
            break
train_fr_sub_raw = train_fr_sub_raw[:limit_data_len]
# 데이터 확인
print(len(train_fr_sub_raw))
print(train_fr_sub_raw[:10])

108640
['"Adieu, Valentine, adieux!', 'Prouve, Danglars.', 'Dit Franz.', 'Dit Franz.', '"" Fernand Mondego.', '"" Et le couloir?', '"" Pour Andrea Rondolo?', '"" Pour Andrea Rondolo?', '"A déclaré Fernand.', '"Dit Andrea.']


In [155]:
def filter_long_sentences(french_data, english_data, max_french_length):
    filtered_data = []
    for fr, en in zip(french_data, english_data):
        if len(fr.split()) <= max_french_length:
            filtered_data.append((fr, en))
    return filtered_data

# 최대 프랑스어 문장 길이
max_french_length = 350

# 문장 길이가 최대값을 초과하는 문장 제거
filtered_data = filter_long_sentences(train_fr_sub_raw, train_en_sub_raw, max_french_length)

# 새로운 데이터셋 생성
train_fr_sub_raw, train_en_sub_raw = zip(*filtered_data)

print(len(train_en_sub_raw))
print(len(train_fr_sub_raw))
print(train_en_sub_raw[:10])
print(train_fr_sub_raw[:10])

108639
108639
('ADIEU VALENTINE ADIEU', 'PROVE IT DANGLARS', 'SAID FRANZ', 'SAID FRANZ', 'FERNAND MONDEGO', 'AND THE CORRIDOR', 'FOR ANDREA RONDOLO', 'FOR ANDREA RONDOLO', 'SAID FERNAND', 'SAID ANDREA')
('"Adieu, Valentine, adieux!', 'Prouve, Danglars.', 'Dit Franz.', 'Dit Franz.', '"" Fernand Mondego.', '"" Et le couloir?', '"" Pour Andrea Rondolo?', '"" Pour Andrea Rondolo?', '"A déclaré Fernand.', '"Dit Andrea.')


In [156]:
with open(test_eng_path, "r", encoding="utf-8") as file:
    test_en_raw = file.readlines()
    
with open(test_de_path, "r", encoding="utf-8") as file:
    test_fr_raw = file.readlines()

with open(test_dev_eng_path, "r", encoding="utf-8") as file:
    test_dev_en_raw = file.readlines()
    
with open(test_dev_de_path, "r", encoding="utf-8") as file:
    test_dev_fr_raw = file.readlines()

print(test_en_raw[:2])
print(test_fr_raw[:2])

print(len(test_en_raw))
print(len(test_fr_raw))
test_en_raw += test_dev_en_raw
test_fr_raw += test_dev_fr_raw

['IN ANOTHER THE GROUND WAS CUMBERED WITH RUSTY IRON MONSTERS OF STEAM BOILERS WHEELS CRANKS PIPES FURNACES PADDLES ANCHORS DIVING BELLS WINDMILL SAILS AND I KNOW NOT WHAT STRANGE OBJECTS ACCUMULATED BY SOME SPECULATOR AND GROVELLING IN THE DUST UNDERNEATH WHICH HAVING SUNK INTO THE SOIL OF THEIR OWN WEIGHT IN WET WEATHER THEY HAD THE APPEARANCE OF VAINLY TRYING TO HIDE THEMSELVES\n', 'THE CLASH AND GLARE OF SUNDRY FIERY WORKS UPON THE RIVER SIDE AROSE BY NIGHT TO DISTURB EVERYTHING EXCEPT THE HEAVY AND UNBROKEN SMOKE THAT POURED OUT OF THEIR CHIMNEYS\n']
["Dans un autre, le terrain était encombré de monstres en fer rouillé de chaudières à vapeur, de roues, de manivelles, de tuyaux, de fours, de pagaies, d'ancrages, de cloches, de voiliers, et je ne sais pas quels objets étranges, accumulés par un spéculateur, et Se creusant dans la poussière, sous laquelle - s'étant enfoncée dans le sol de leur propre poids par temps humide - ils avaient l'apparence d'essayer vainement de se cacher.\n

In [157]:
from unicodedata import normalize

def unicodeToAscii(s):
    return normalize('NFD', s).encode('ascii', 'ignore').decode('utf-8')

In [158]:
import unicodedata
from unicodedata import normalize
import re

def normalizeString(s):
    # s = unicodeToAscii(s.lower().strip())
    s = unicodeToAscii(s)
    # s = re.sub(r"([.!?\"])", r" \1", s) # 마침표, 느낌표, 물음표, 따옴표 앞에 공백을 추가
    # s = re.sub(r"[^a-zA-Z.!?]+", r" ", s) # G 영문 알파벳과 마침표(.), 느낌표(!), 물음표(?)를 제외한 모든 문자를 공백으로 대체/ +: 해당 패턴이 한 번 이상 반복
    s = re.sub(r"[^a-zA-Z]+", r" ", s) # 이 코드 실행 후 strip() 안하면 문장 앞뒤 제거됐을 때 공백생김 
    return s.strip()

print(train_fr_sub_raw[2222]) 
print(normalizeString(train_fr_sub_raw[2222]))

print(train_fr_sub_raw[3332]) 
print(normalizeString(train_fr_sub_raw[3332]))

Oh, Beauchamp, Beauchamp, comment puis-je aborder le mien?
Oh Beauchamp Beauchamp comment puis je aborder le mien
Mais, après m'avoir péché, peut-être plus profondément que d'autres, je ne me reposerai jamais avant d'avoir déchiré les semblables de mes semblables et j'ai découvert leurs faiblesses. Je les ai toujours trouvés; Et plus encore, je le répète avec joie, avec triomphe, j'ai toujours trouvé une preuve de perversité humaine ou d'erreur.
Mais apres m avoir peche peut etre plus profondement que d autres je ne me reposerai jamais avant d avoir dechire les semblables de mes semblables et j ai decouvert leurs faiblesses Je les ai toujours trouves Et plus encore je le repete avec joie avec triomphe j ai toujours trouve une preuve de perversite humaine ou d erreur


In [159]:
def normalizeStrings(lines):
    return [normalizeString(s) for s in lines]
    
train_fr_sub = normalizeStrings(train_fr_sub_raw)
train_en_sub = normalizeStrings(train_en_sub_raw)
test_fr = normalizeStrings(test_fr_raw)
test_en = normalizeStrings(test_en_raw)

In [160]:
print(train_fr_sub[100:105])
print(train_fr_sub_raw[100:105])

['Le celebre Cucumetto poursuivi dans les Abruzzes chasse du royaume de Naples ou il avait mene une guerre reguliere avait traverse le Garigliano comme Manfred et s etait refugie sur les rives de l Amasine entre Sonnino et Juperno', 'Vous connaissez les environs de Paris alors', 'Vous connaissez les environs de Paris alors', 'Franz prit la lampe et entra dans la grotte souterraine suivie de Gaetano', 'Gaetano ne se trompait pas']
('"Le célèbre Cucumetto, poursuivi dans les Abruzzes, chassé du royaume de Naples, où il avait mené une guerre régulière, avait traversé le Garigliano, comme Manfred, et s\'était réfugié sur les rives de l\'Amasine entre Sonnino et Juperno.', '"" Vous connaissez les environs de Paris, alors?', '"" Vous connaissez les environs de Paris, alors?', 'Franz prit la lampe et entra dans la grotte souterraine, suivie de Gaetano.', 'Gaetano ne se trompait pas.')


In [161]:
from torchtext.data import Dataset, Example

def create_dataset(text_list, field_name, field_obj):
    # G Example.fromlist(data_list, fields) : field 정의에 따라 데이터를 적절히 처리하고 Example 객체를 생성
    #   data_list의 순서는 필드 정의에서 지정한 순서와 일치해야 합니다.
    examples = [Example.fromlist([text], [(field_name, field_obj)]) for text in text_list]
    return Dataset(examples, fields=[(field_name, field_obj)])

리브리스피치는 아래 부분 7_mydata로 변경

In [162]:
from torchtext.vocab import FastText, GloVe

flag_pretrained_emb = False

train_data = create_dataset(train_fr_sub, 'src', SRC)

train_data.fields['trg'] = TRG
train_en_data = create_dataset(train_en_sub, 'trg', TRG)

for i in range(len(train_data)):
    train_data[i].trg = train_en_data[i].trg

test_data = create_dataset(test_fr, 'src', SRC)

test_data.fields['trg'] = TRG
test_en_path = create_dataset(test_en, 'trg', TRG)

for i in range(len(test_data)):
    test_data[i].trg = test_en_path[i].trg

max_vocab_size = 100000 # 12000
if flag_pretrained_emb :
    # print("loading en_pretrained_emb. . .")
    # en_pretrained_emb = GloVe(name='6B', dim=300, cache='/data/hwyu/1_seq2seq/cache_dir_en')

    print("loading en_fasttext. . .")
    en_pretrained_emb = torch.load('/data/hwyu/1_seq2seq/en_fasttext_dim300.pt')
    print("load 완료\n")

    TRG.build_vocab(train_data, max_size=max_vocab_size, min_freq=2, vectors=en_pretrained_emb)
else :
    TRG.build_vocab(train_data, max_size=max_vocab_size, min_freq=2) # min_freq=3

In [163]:
print(f"학습 데이터셋(training dataset) 크기: {len(train_data.examples)}개")
# print(f"평가 데이터셋(validation dataset) 크기: {len(valid_data.examples)}개")
print(f"테스트 데이터셋(testing dataset) 크기: {len(test_data.examples)}개")

학습 데이터셋(training dataset) 크기: 108639개
테스트 데이터셋(testing dataset) 크기: 3119개


In [164]:
# 학습 데이터 중 하나를 선택해 출력
# ein ~ 문장이 들어 왔을 때 a ~ 이러한 영어 문장을 출력하도록 학습 데이터가 구성됨
# G vars() : 내장 함수 중 하나로, 인자가 객체인 경우 해당 객체의 __dict__ 속성 반환
    # __dict__ : 이 사전에는 객체가 가지고 있는 모든 속성과 해당 값 포함
    # vars(obj)  # {'x': 10, 'y': 20}
        # x, y : obj 인스턴스의 멤버변수
    # 인자 없이 호출 : 현재 스코프에서 정의된 모든 변수와 그 값
print(vars(train_data.examples[30])['src'])
print(vars(train_data.examples[30])['trg'])

[137, 4428, 55, 24968]
['asked', 'franz']


* **필드(field)** 객체의 **build_vocab** 메서드를 이용해 영어와 독어의 단어 사전을 생성합니다.
  * 초기 input 디멘젼을 구하기 위해 
  * **최소 2번 이상** 등장한 단어만을 선택합니다.

In [165]:
# SRC.build_vocab(train_dataset, min_freq=2)
# TRG.build_vocab(train_data, min_freq=2)
print(TRG.vocab.freqs.most_common(10))

# print(f"len(SRC): {len(SRC.vocab)}") # 독일어는 7855개의 유의미한 단어가 있음
print(f"len(TRG): {len(TRG.vocab)}")

[('the', 126315), ('and', 70406), ('of', 66454), ('to', 59299), ('a', 46152), ('in', 35949), ('i', 33435), ('he', 29094), ('that', 28312), ('was', 25626)]
len(TRG): 29142


In [166]:
# stoi : 해당 단어의 인덱스 
print(TRG.vocab.stoi["abcabc"]) # 없는 단어: 0 ;unk
print(TRG.vocab.stoi[TRG.pad_token]) # 패딩(padding): 1
print(TRG.vocab.stoi["<sos>"]) # <sos>: 2
print(TRG.vocab.stoi["<eos>"]) # <eos>: 3
print(TRG.vocab.stoi["hello"]) # 단어의 인덱스 출력
print(TRG.vocab.stoi["world"])

0
1
2
3
15039
200


In [167]:
print(TRG.eos_token, TRG.init_token, TRG.pad_token, TRG.unk_token)
print(SRC.eos_token, SRC.init_token, SRC.pad_token, SRC.unk_token)

<eos> <sos> <pad> <unk>
6 5 1 3


In [168]:
if flag_pretrained_emb :
    print("\n임베딩 벡터 확인:")
    print("English Embedding Shape:", TRG.vocab.vectors.shape)
    
    print("\n임베딩 벡터 값 확인:")
    print("English Embedding Vectors:")
    print(TRG.vocab.vectors)
    
    # G 특별 토큰들에 대한 임베딩은 훈련되지 않습니다. 
    #  따라서 초기화 단계에서는 이러한 토큰들의 임베딩은 보통 0으로 설정됩니다.
    # print(torch.all(TRG.vocab.vectors[0] == TRG.vocab.vectors[1]))
    # print(torch.all(TRG.vocab.vectors[1] == TRG.vocab.vectors[2]))
    # print(torch.all(TRG.vocab.vectors[2] == TRG.vocab.vectors[3]))
    # print(torch.all(TRG.vocab.vectors[3] == TRG.vocab.vectors[4]))

In [169]:
if flag_pretrained_emb :
    def count_zero_rows(matrix):
        count = 0
        for row in matrix:
            if all(element == 0 for element in row):
                count += 1
        return count
    
    print("0으로만 이루어진 행의 개수:", count_zero_rows(TRG.vocab.vectors)) 

In [170]:
if flag_pretrained_emb :
    trg_embedding = torch.nn.Embedding.from_pretrained(TRG.vocab.vectors) # freeze=True # 디폴트
    print(trg_embedding.weight.requires_grad)
    
    # 임베딩 레이어의 가중치 확인:
    print("\nEnglish Embedding Layer Weights:")
    print(trg_embedding.weight)

In [171]:
from pprint import pprint

print(f"Number of training examples: {len(train_data.examples)}")
# print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

print(train_data[5].__dict__.keys())
print(f"txt 파일 : {train_fr_sub[5]}")
print(f"txt 파일 : {train_en_sub[5]}")
# data 딕셔너리의 내용이 보기 좋게 출력
pprint(train_data[5].__dict__.values()) # @나는 콤마 뒤는 잘림 .tok 파일은 다 자르나? 아니다  format='csv'로 해서 그런거였음  

Number of training examples: 108639
Number of testing examples: 3119
dict_keys(['src', 'trg'])
txt 파일 : Et le couloir
txt 파일 : AND THE CORRIDOR
dict_values([[139, 16, 9144], ['and', 'the', 'corridor']])


In [172]:
pprint(train_data[36].__dict__.values())
print(f"txt 파일 : {train_fr_sub[36]}")
print(f"txt 파일 : {train_en_sub[36]}")

dict_values([[137, 4428, 55, 137, 7211, 105], ['asked', 'debray']])
txt 파일 : Demanda Debray
txt 파일 : ASKED DEBRAY


In [173]:
print(test_data[113].__dict__.keys())
pprint(test_data[113].__dict__.values())
print(f"txt 파일 : {test_fr[113]}")
print(f"txt 파일 : {test_en[113]}")

dict_keys(['src', 'trg'])
dict_values([[51, 33, 18553], ['he', 'demanded']])
txt 파일 : il a ordonne
txt 파일 : HE DEMANDED


In [174]:
print(tokenizer.batch_decode([[137, 4428, 55, 137, 7211, 105]]))

['Demanda Debray']


* 한 문장에 포함된 단어가 순서대로 나열된 상태로 네트워크에 입력되어야 합니다.
    * ~따라서 하나의 배치에 포함된 문장들이 가지는 단어의 개수가 유사하도록 만들면 좋습니다.~
    * 이를 위해 BucketIterator를 사용합니다.
      - 다른 길이의 시퀀스를 효율적으로 묶어서 배치하는 데이터 이터레이터의 한 유형
      - 버킷 : 시퀀스를 길이별로 정렬하여 서로 다른 "버킷"이나 그룹으로 나누는 과정
      - 훈련 중에는 이러한 버킷에서 시퀀스를 선택하여 배치를 형성
    * **배치 크기(batch size)**: 128

In [175]:
BATCH_SIZE = 128

train_iterator, test_iterator = BucketIterator.splits(
    (train_data, test_data),
    batch_size=BATCH_SIZE, # 디폴트 1
    # sort : 데이터를 정렬할지 여부를 지정하는 인자, 기본값은 True
    # sort_within_batch=True, # 기본값은 False / ?? 안해도 되나 -> sort 인자가 트루면 각 배치 내에서 데이터를 sort_key에 따라 다시 정렬
    sort_key=lambda x: len(x.src), # G 유사한 길이의 문장을 같은 배치로 묶기 위해
        # 안하면 evaluate() 실행시 TypeError: '<' not supported between instances of 'Example' and 'Example'
        # 근데 왜 train() 실행 시에는 에러 안나는 거지 
    device=device)

In [176]:
count = 0
max_len_eng = []
max_len_ger = []
for data in train_data:
  max_len_ger.append(len(data.src))
  max_len_eng.append(len(data.trg))
  if count < 3 :
    print("German - ",*data.src, " Length - ", len(data.src))
    print("English - ",*data.trg, " Length - ", len(data.trg))
    print()
  count += 1

print("Maximum Length of English sentence {} and German sentence {} in the dataset".format(max(max_len_eng),max(max_len_ger)))
print("Minimum Length of English sentence {} and German sentence {} in the dataset".format(min(max_len_eng),min(max_len_ger)))

bert_max_len = tokenizer.model_max_length
assert max(max_len_ger) <= bert_max_len, "버트 최대 입력 길이 초과"

German -  114 13036 11407 35 22934 290  Length -  6
English -  adieu valentine adieu  Length -  3

German -  1092 8304 160 1907 5481 10  Length -  6
English -  prove it danglars  Length -  3

German -  21287 24968  Length -  2
English -  said franz  Length -  2

Maximum Length of English sentence 272 and German sentence 395 in the dataset
Minimum Length of English sentence 1 and German sentence 0 in the dataset


In [177]:
# # 첫번째 배치에서 하나의 문장 정보 출력
# for i, batch in enumerate(train_iterator):
#     src = batch.src
#     trg = batch.trg

#     print(f"첫 번째 배치 크기: {src.shape}") # 128개의 문장 중 가장 긴 문장의 길이가 35

#     for i in range(0, 2): 
#         print(src[i])

#     # 첫 번째 배치만 확인
#     break

In [178]:
# 첫번째 배치에서 하나의 문장 정보 출력
for i, batch in enumerate(train_iterator):
    src = batch.src
    print(src[:2])

    attention_mask = (src != SRC.pad_token).type(torch.long)
    print(attention_mask[:2]) #okenizer.sep_token_id
    
    break

tensor([[    5,    84,    14,   199,    23,   393,   421,  2812,   670,     8,
           741,  1173,    42,    20,  6308, 14111,    14, 17305,     6,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1],
        [    5,   156,    21, 28089,    99,     8,   127,  1401,    14,   488,
            28, 21842,    79,    86,    13,  5099,     8,    65,    52,    21,
         27151,   312,    33,    13,  3385,  4162,  2068,    19,   529,    56,
            16

In [179]:
for i, batch in enumerate(train_iterator):
    src = batch.src[:2]

    last_hidden_state = bert_model(src).last_hidden_state
    
    # [CLS] 및 [SEP] 토큰의 인덱스 가져오기
    # cls_index = 0
    cls_indices = torch.tensor([0])  # [CLS] 토큰의 인덱스
    sep_indices = (src == tokenizer.sep_token_id).nonzero()[:, 1]  # [SEP] 토큰의 인덱스
    
    # 각 배치에 대해 [CLS] 및 [SEP] 토큰의 임베딩을 0으로 설정
    for batch_idx in range(last_hidden_state.size(0)):
        last_hidden_state[batch_idx, cls_indices, :] = 0  # [CLS] 토큰의 임베딩을 0으로 설정
        last_hidden_state[batch_idx, sep_indices[batch_idx], :] = 0  # [SEP] 토큰의 임베딩을 0으로 설정
    
    for i in range(src.size(0)):
        print("[CLS]: ", last_hidden_state[i,cls_indices,:10])
        print("[SEP]: ", last_hidden_state[i,sep_indices[i],:10], '\n')

    break

[CLS]:  tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], device='cuda:0',
       grad_fn=<IndexBackward0>)
[SEP]:  tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0',
       grad_fn=<SliceBackward0>) 

[CLS]:  tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], device='cuda:0',
       grad_fn=<IndexBackward0>)
[SEP]:  tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0',
       grad_fn=<SliceBackward0>) 



#### **Multi Head Attention 아키텍처**

* 어텐션(attention)은 <b>세 가지 요소</b>를 입력으로 받습니다.
    * <b>쿼리(queries)</b>, <b>키(keys)</b>, <b>값(values)</b>
    * 현재 구현에서는 Query, Key, Value의 차원이 모두 같습니다.
* 하이퍼 파라미터(hyperparameter)
    * **hidden_dim**: 하나의 단어에 대한 임베딩 차원
    * **n_heads**: 헤드(head)의 개수 = scaled dot-product attention의 개수
    * **dropout_ratio**: 드롭아웃(dropout) 비율

**멀티 헤드 어텐션을 구현 :**

1. 어텐션 헤드 수 정의

1. 프로젝션 수행: 입력 벡터를 여러 헤드로 분할하기 위해 선형 프로젝션을 수행합니다. 이는 입력 벡터를 각 어텐션 헤드의 차원으로 분할하는 과정입니다.

1. 각 헤드에 대한 어텐션 계산: 분할된 입력 벡터를 각 어텐션 헤드에 대해 어텐션을 계산합니다. 각 헤드에서는 각자의 쿼리(Q), 키(K), 값(V)에 대한 어텐션을 계산합니다.

1. 헤드 결합: 각 헤드에서 계산된 어텐션 값을 결합하여 최종 어텐션 값으로 합칩니다. 이렇게 하면 각 헤드가 다르게 강조한 정보를 결합하여 하나의 표현으로 얻을 수 있습니다.
- ~??~ 멀티헤드 쓰려면 임베딩 차원을 쪼개니까 병령 처리로 속도만 빨라지고 아무리 다양한 각도로 본다해도 쪼개면 부분적으로만 보니까 직관적으로 생각해봤을 때는 오히려 성능이 더 떨어지지 않나?
    -  다른 시각으로 볼 수 있다는 메리트가 더 큰 듯

In [180]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hidden_dim, n_heads, dropout_ratio, device):
        super().__init__()

        # hidden_dim이 n_heads로 나누어 떨어지지 않으면 AssertionError를 발생시킴
        # 이때 은닉 차원을 어텐션 헤드의 수로 나누어 떨어지게 하는 것은 헤드 간에 정보를 공유하고 연산을 병렬화하는 데 도움이 됨
        assert hidden_dim % n_heads == 0

        self.hidden_dim = hidden_dim # 임베딩 차원
        self.n_heads = n_heads # 헤드(head)의 개수: 서로 다른 어텐션(attention) 컨셉의 수
        self.head_dim = hidden_dim // n_heads # 각 헤드(head)에서의 임베딩 차원   / fc_q의 결과 디멘젼을 n_heads개로 쪼개서 사용

        # 쿼리(Q)를 계산하기 위한 레이어 ;입력을 쿼리로 변환하기 위한 레이어
        self.fc_q = nn.Linear(hidden_dim, hidden_dim) # Query 값에 적용될 FC 레이어
        self.fc_k = nn.Linear(hidden_dim, hidden_dim) # Key 값에 적용될 FC 레이어
        self.fc_v = nn.Linear(hidden_dim, hidden_dim) # Value 값에 적용될 FC 레이어

        # 어텐션 메커니즘에서 사용되는 출력을 계산하기 위한 레이어
        # : Q,K,V를 이용하여 어텐션을 계산한 후, 이를 조합하여 최종 출력을 생성
          # 이 과정에서 self.fc_o는 어텐션 값을 변환하여 최종 출력을 계산
        self.fc_o = nn.Linear(hidden_dim, hidden_dim)

        self.dropout = nn.Dropout(dropout_ratio)

        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)

    def forward(self, query, key, value, mask = None):

        batch_size = query.shape[0]

        # query: [batch_size, query_len, hidden_dim]   / query_len : 단어 개수
        # key: [batch_size, key_len, hidden_dim]
        # value: [batch_size, value_len, hidden_dim]

        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)

        # Q: [batch_size, query_len, hidden_dim]
        # K: [batch_size, key_len, hidden_dim]
        # V: [batch_size, value_len, hidden_dim]

        # hidden_dim → n_heads X head_dim 형태로 변형
        # n_heads(h)개의 서로 다른 어텐션(attention) 컨셉을 학습하도록 유도
        #   Q,K,V 결과 값을 h개로 나눠 사용 ;n_heads개 각각 마다 head_dim 만큼의 크기로 차원을 가지도록 만들어서 ;h개의 Q,K,V 만듦
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        # ?? view(batch_size, self.n_heads, -1, self.head_dim) 이렇게 하면 permute 안해도 되지 않나
        
        # Q: [batch_size, n_heads, query_len, head_dim]
        # K: [batch_size, n_heads, key_len, head_dim]
        # V: [batch_size, n_heads, value_len, head_dim]

        # Attention Energy 계산
        # 각 head마다 Q,K 서로 곱하고 scale로 나눠서 energy 구함
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale

        # energy: [batch_size, n_heads, query_len, key_len]

        # 마스크(mask)를 사용하는 경우
        if mask is not None:
            # 마스크(mask) 값이 0인 부분을 -1e10으로 채우기 -> softmax에 들어간 값이 거의 0%가 나오게
            energy = energy.masked_fill(mask==0, -1e10)

        # 어텐션(attention) 스코어 계산: 각 단어에 대한 확률 값
        attention = torch.softmax(energy, dim=-1) # 마지막 차원을 따라 소프트맥스 연산 수행
        # @ 각 행의 합이 1
        
        # attention: [batch_size, n_heads, query_len, key_len]

        # 여기에서 Scaled Dot-Product Attention을 계산
        # 위에서 softmax를 취해서 나온 attention 가중치 * V 해서 어텐션 밸값을 결과적으로 만들어줌
        x = torch.matmul(self.dropout(attention), V)

        # x: [batch_size, n_heads, query_len, head_dim]

        x = x.permute(0, 2, 1, 3).contiguous() # contiguous : 메모리 레이아웃을 연속적으로 만들어줌 -> 행렬 연산을 효율적으로 수행하기 위해
        # ?? 왜 permute -> 다음 코드에서 n_heads, head_dim를 hidden_dim로 연결하기 위해
        
        # x: [batch_size, query_len, n_heads, head_dim]

        x = x.view(batch_size, -1, self.hidden_dim) # ;일자로 쭉 늘어뜨림; concat

        # x: [batch_size, query_len, hidden_dim]

        x = self.fc_o(x)

        # x: [batch_size, query_len, hidden_dim]

        return x, attention # 나중에 시각화도 하려고 attention 스코어값도 출력

#### **Position-wise Feedforward 아키텍처**

* 입력과 출력의 차원이 동일합니다.
* 하이퍼 파라미터(hyperparameter)
    * **hidden_dim**: 하나의 단어에 대한 임베딩 차원
    * **pf_dim**: Feedforward 레이어에서의 내부 임베딩 차원
    * **dropout_ratio**: 드롭아웃(dropout) 비율

In [181]:
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hidden_dim, pf_dim, dropout_ratio):
        super().__init__()

        # hidden_dim -> hidden_dim : 입출력 차원 동일
        self.fc_1 = nn.Linear(hidden_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hidden_dim)

        self.dropout = nn.Dropout(dropout_ratio)

    def forward(self, x):

        # x: [batch_size, seq_len, hidden_dim]

        x = self.dropout(torch.relu(self.fc_1(x)))

        # x: [batch_size, seq_len, pf_dim]

        x = self.fc_2(x)

        # x: [batch_size, seq_len, hidden_dim]

        return x

In [182]:
class DecoderLayer(nn.Module):
    def __init__(self, hidden_dim, n_heads, pf_dim, dropout_ratio, device):
        super().__init__()

        # 총 6개 레이어 <- 디코더가 이렇게 구성됨. 이걸 여러번 중첩
        self.self_attn_layer_norm = nn.LayerNorm(hidden_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hidden_dim)
        self.ff_layer_norm = nn.LayerNorm(hidden_dim)
        self.self_attention = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio, device)
        self.encoder_attention = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hidden_dim, pf_dim, dropout_ratio)
        self.dropout = nn.Dropout(dropout_ratio)

    # 인코더의 출력 값(enc_src)을 어텐션(attention)하는 구조
    def forward(self, trg, enc_src, trg_mask, src_mask):

        # trg: [batch_size, trg_len, hidden_dim]
        # enc_src: [batch_size, src_len, hidden_dim]
        # trg_mask: [batch_size, trg_len] 
        # src_mask: [batch_size, src_len]
        # mask 크기 잘 못 쓴 듯 : 두 코드 참고
            # trg_mask: [batch_size, 1, trg_len, trg_len]
            # energy.masked_fill(mask==0, -1e10)

        # 1. self attention
        # 자기 자신에 대하여 어텐션(attention)
        # 1~3rd : q,k,v는 모두 자기 자신(trg)을 넣어서 만듦
        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)

        # 2. dropout, residual connection and layer norm
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))

        # trg: [batch_size, trg_len, hidden_dim]

        # 3. encoder attention
        # 디코더의 쿼리(Query)를 이용해 인코더를 어텐션(attention)
        # 인코더 디코더 어텐션 수행 : 인코더에서 정보를 가져옴
        # Q - trg : 디코드에 포함되어 있는 출력 단어들에 대한 정보
        # K - enc_src : 인코더에서 가장 마지막 출력 값으로 나온 값
        _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)

        # dropout, residual connection and layer norm
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))

        # trg: [batch_size, trg_len, hidden_dim]

        # positionwise feedforward
        _trg = self.positionwise_feedforward(trg)

        # dropout, residual and layer norm
        trg = self.ff_layer_norm(trg + self.dropout(_trg))

        # trg: [batch_size, trg_len, hidden_dim]
        # attention: [batch_size, n_heads, trg_len, src_len]

        return trg, attention

In [183]:
if flag_pretrained_emb :
    HIDDEN_DIM = len(trg_embedding.weight[1])
else:
    HIDDEN_DIM = 256 #
print(HIDDEN_DIM)

256


In [184]:
BERT_HIDDEN_DIM = bert_model.config.hidden_size

In [185]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_layers, n_heads, pf_dim, dropout_ratio, device, max_length=500): # max_length - change
        super().__init__()

        # self.device = device
        # self.dropout = nn.Dropout(dropout_ratio)
        # self.scale = torch.sqrt(torch.FloatTensor([hidden_dim])).to(device)
        # self.bert = bert_model

    def forward(self, src):

        # src: [batch_size, src_len]
        
        attention_mask = (src != SRC.pad_token).type(torch.long)
        with torch.no_grad():
            enc_src = bert_model(src, attention_mask=attention_mask)[0][:,:,:HIDDEN_DIM]

        # enc_src: [batch_size, src_len, hidden_dim]
        
        cls_indices = torch.tensor([0])  # [CLS] 토큰의 인덱스
        sep_indices = (src == tokenizer.sep_token_id).nonzero()[:, 1]  # [SEP] 토큰의 인덱스
        
        # 각 배치에 대해 [CLS] 및 [SEP] 토큰의 임베딩을 0으로 설정
        for batch_idx in range(enc_src.size(0)):
            enc_src[batch_idx, cls_indices, :] = 0  # [CLS] 토큰의 임베딩을 0으로 설정
            enc_src[batch_idx, sep_indices[batch_idx], :] = 0  

        return enc_src # 마지막 레이어의 출력을 반환

In [186]:
class Decoder(nn.Module):
    def __init__(self, output_dim, hidden_dim, n_layers, n_heads, pf_dim, dropout_ratio, device, max_length=max(max_len_ger)):
        super().__init__()

        self.device = device

        # output_dim : 단어 개수
        if flag_pretrained_emb :
            self.tok_embedding = trg_embedding
        else :        
            self.tok_embedding = nn.Embedding(output_dim, hidden_dim)        # max_length : seq len 
        self.pos_embedding = nn.Embedding(max_length, hidden_dim)

        self.fc1 = nn.Linear(hidden_dim, BERT_HIDDEN_DIM)
        self.fc2 = nn.Linear(BERT_HIDDEN_DIM, hidden_dim)

        self.layers = nn.ModuleList([DecoderLayer(hidden_dim, n_heads, pf_dim, dropout_ratio, device) for _ in range(n_layers)])

        self.fc_out = nn.Linear(hidden_dim, output_dim)

        self.dropout = nn.Dropout(dropout_ratio)

        self.scale = torch.sqrt(torch.FloatTensor([hidden_dim])).to(device)

    def forward(self, trg, enc_src, trg_mask, src_mask):

        # trg: [batch_size, trg_len] /타겟 문장에 대한 정보
        # enc_src: [batch_size, src_len, hidden_dim] /인코더 마지막 출력값
        # trg_mask: [batch_size, trg_len]
        # src_mask: [batch_size, src_len]

        batch_size = trg.shape[0]
        trg_len = trg.shape[1]

        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)

        # pos: [batch_size, trg_len]

        trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))

        # trg: [batch_size, trg_len, hidden_dim]

        for layer in self.layers:
            # 소스 마스크와 타겟 마스크 모두 사용
            # trg, attention = layer(trg, enc_src, trg_mask, src_mask) 
            # trg, attention = layer(trg, self.fc2(self.fc1(enc_src)), trg_mask, src_mask) 
            
            trg, attention = layer(trg, self.fc2(self.dropout(torch.relu((self.fc1(enc_src))))), trg_mask, src_mask)


        # trg: [batch_size, trg_len, hidden_dim]
        # attention: [batch_size, n_heads, trg_len, src_len]

        # 출력을 위한 fc 거침
        output = self.fc_out(trg)

        # output: [batch_size, trg_len, output_dim]

        return output, attention

In [187]:
tokenizer.tokenize('il etudie leurs habitudes')

['▁il', '▁et', 'udi', 'e', '▁leurs', '▁habitudes']

#### **트랜스포머(Transformer) 아키텍처**

* 최종적인 전체 트랜스포머(Transformer) 모델을 정의합니다.
* 입력이 들어왔을 때 앞서 정의한 인코더와 디코더를 거쳐 출력 문장을 생성합니다.

In [188]:
class Transformer(nn.Module):
    def __init__(self, encoder, decoder, src_pad_idx, trg_pad_idx, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    # 소스 문장의 <pad> 토큰에 대하여 마스크(mask) 값을 0으로 설정
    def make_src_mask(self, src):

        # src: [batch_size, src_len]

        # G () : 패딩 토큰과 일치하지 않는 위치를 찾음
        #   unsqueeze(2) : 브로드캐스팅(broadcasting) 연산을 수행할 수 있도록
        #   src_mask : 패딩 토큰을 포함하지 않는 입력 시퀀스의 위치에는 True
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

        # src_mask: [batch_size, 1, 1, src_len]

        return src_mask

    # 타겟 문장에서 각 단어는 다음 단어가 무엇인지 알 수 없도록(이전 단어만 보도록) 만들기 위해 마스크를 사용
    def make_trg_mask(self, trg):

        # trg: [batch_size, trg_len]

        # 1. 소스 문장과 동일하게 pad 마스킹  
        """ (마스크 예시)
        1 0 0 0 0
        1 1 0 0 0
        1 1 1 0 0
        1 1 1 0 0
        1 1 1 0 0
        """
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)

        # trg_pad_mask: [batch_size, 1, 1, trg_len]

        trg_len = trg.shape[1]

        # 2. 별도의 마스크 하나 더 만듦 : 앞쪽 단어만 볼 수 있게
        """ (마스크 예시)
        1 0 0 0 0
        1 1 0 0 0
        1 1 1 0 0
        1 1 1 1 0
        1 1 1 1 1
        """
        # G .tril()은 행렬의 하삼각 부분을 반환. 모두 1로 채워진 행렬에 적용
          # (trg_len, trg_len)은 정사각형 행렬을 생성하기 위해
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()

        # trg_sub_mask: [trg_len, trg_len]

        # G 브로드캐스팅은
          # 두 배열의 차원 수가 다르면 차원 수가 더 적은 배열의 형상이 더 많은 배열의 형상에 맞춰지도록 자동으로 확장
          # 두 배열의 크기가 어느 한 차원에서 일치하지 않으면 크기가 1인 차원이 다른 배열의 크기에 맞추어 확장
        trg_mask = trg_pad_mask & trg_sub_mask

        # trg_mask: [batch_size, 1, trg_len, trg_len]

        return trg_mask

    def forward(self, src, trg):

        # src: [batch_size, src_len]
        # trg: [batch_size, trg_len]

        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)

        # src_mask: [batch_size, 1, 1, src_len]
        # trg_mask: [batch_size, 1, trg_len, trg_len]

        # enc_src = self.encoder(src, src_mask)
        enc_src = self.encoder(src)
        
        # output : 번역 결과
        # 디코더는 매번 enc_src(인코더의 출력값)을 어텐션
        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)

        # output: [batch_size, trg_len, output_dim]
        # attention: [batch_size, n_heads, trg_len, src_len]

        return output, attention

In [189]:
vocab_size = tokenizer.vocab_size
print("어휘 사전의 크기:", vocab_size)

어휘 사전의 크기: 32000


#### **학습(Training)**

* 하이퍼 파라미터 설정 및 모델 초기화

In [190]:
INPUT_DIM = 0 # 아직 사용x
OUTPUT_DIM = len(TRG.vocab)
ENC_LAYERS = 3 # 아직 사용x
DEC_LAYERS = 3
ENC_HEADS = 8 # 아직 사용x
DEC_HEADS = 8
ENC_PF_DIM = 512 # 아직 사용x
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1 # 아직 사용x
DEC_DROPOUT = 0.1

In [191]:
# G SRC 텍스트 필드에서 pad_token 속성을 사용하여 패딩 토큰의 문자열을 가져온 다음, stoi (string to index) 속성을 사용하여 해당 문자열을 정수 인덱스로 변환
SRC_PAD_IDX = SRC.pad_token
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

# 인코더(encoder)와 디코더(decoder) 객체 선언
enc = Encoder(INPUT_DIM, HIDDEN_DIM, ENC_LAYERS, ENC_HEADS, ENC_PF_DIM, ENC_DROPOUT, device)
dec = Decoder(OUTPUT_DIM, HIDDEN_DIM, DEC_LAYERS, DEC_HEADS, DEC_PF_DIM, DEC_DROPOUT, device)

# Transformer 객체 선언
model = Transformer(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

In [192]:
for name, param in model.named_parameters():
    # <는 왼쪽 정렬, 숫자는 해당 필드의 최소 폭
    print("{:<70} {}".format(name, param.requires_grad))

decoder.tok_embedding.weight                                           True
decoder.pos_embedding.weight                                           True
decoder.fc1.weight                                                     True
decoder.fc1.bias                                                       True
decoder.fc2.weight                                                     True
decoder.fc2.bias                                                       True
decoder.layers.0.self_attn_layer_norm.weight                           True
decoder.layers.0.self_attn_layer_norm.bias                             True
decoder.layers.0.enc_attn_layer_norm.weight                            True
decoder.layers.0.enc_attn_layer_norm.bias                              True
decoder.layers.0.ff_layer_norm.weight                                  True
decoder.layers.0.ff_layer_norm.bias                                    True
decoder.layers.0.self_attention.fc_q.weight                            True
decoder.laye

* **모델 가중치 파라미터 초기화**

In [193]:
def count_parameters(model):
    # G numel() : 해당 텐서의 총 요소 수 반환
    #   다차원 텐서의 경우 모든 차원의 크기를 곱한 값 반환
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 17,817,558 trainable parameters


In [194]:
def initialize_weights(m): # m(모듈): 각 레이어
    # G 차원이 1보다 작으면 해당 텐서가 벡터이며, 일반적으로 이러한 경우에는 Xavier 초기화를 적용x
      # 가중치 텐서의 차원이 1보다 작은 경우는 보통 편향(bias)을 나타냅니다. 대부분의 PyTorch 레이어는 가중치와 편향을 함께 가지고 있습니다
      # Xavier 초기화는 주로 가중치에 대해서만 적용되기 때문에 가중치 텐서의 차원이 1보다 작은 경우 Xavier 초기화를 적용하지 않는 것이 일반적
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        # G Xavier 초기화는 각 가중치를 평균이 0이고 분산이 2/(입력 개수 + 출력 개수)인 분포에서 랜덤하게 샘플링하여 초기화
          # 분산을 2/(입력 개수 + 출력 개수)로 설정하는 것은 효율적인 초기화 방법으로, 입력과 출력의 개수가 많은 경우에도 가중치가 적절한 크기로 초기화
        nn.init.xavier_uniform_(m.weight.data)

# G apply : 모델 내의 각 매개변수에 함수를 적용하는 데 사용됩
model.apply(initialize_weights)

Transformer(
  (encoder): Encoder()
  (decoder): Decoder(
    (tok_embedding): Embedding(29142, 256)
    (pos_embedding): Embedding(395, 256)
    (fc1): Linear(in_features=256, out_features=768, bias=True)
    (fc2): Linear(in_features=768, out_features=256, bias=True)
    (layers): ModuleList(
      (0): DecoderLayer(
        (self_attn_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (enc_attn_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (ff_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (self_attention): MultiHeadAttentionLayer(
          (fc_q): Linear(in_features=256, out_features=256, bias=True)
          (fc_k): Linear(in_features=256, out_features=256, bias=True)
          (fc_v): Linear(in_features=256, out_features=256, bias=True)
          (fc_o): Linear(in_features=256, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder_attention): Mult

* 학습 및 평가 함수 정의
    * 기본적인 Seq2Seq 모델과 거의 유사하게 작성할 수 있습니다.

In [195]:
# Adam optimizer로 학습 최적화
LEARNING_RATE = 0.0005
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# 뒷 부분의 패딩(padding)에 대해서는 값 무시
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [196]:
# 모델 학습(train) 함수
from tqdm import tqdm
def train(model, iterator, optimizer, criterion, clip):
    model.train() # 학습 모드
    epoch_loss = 0

    # 전체 학습 데이터를 확인하며
    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg

        optimizer.zero_grad()

        # 출력 단어의 마지막 인덱스(<eos>)는 제외 ?? -> eos는 디코더의 입력이 아니라 디코더의 출력
        # 입력을 할 때는 <sos>부터 시작하도록 처리
        # G 모델은 입력 시퀀스와 이전 부분의 목표 시퀀스를 사용하여 예측을 수행
        output, _ = model(src, trg[:,:-1])

        # output: [배치 크기, trg_len - 1, output_dim]
        # trg: [배치 크기, trg_len]

        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)
        # 출력 단어의 인덱스 0(<sos>)은 제외
        trg = trg[:,1:].contiguous().view(-1)

        # output: [배치 크기 * trg_len - 1, output_dim]
        # trg: [배치 크기 * trg len - 1]

        # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
        loss = criterion(output, trg)
        loss.backward() # 기울기(gradient) 계산

        # 기울기(gradient) clipping 진행
        # G 그래디언트의 전체 노름(norm)을 계산합니다. 그런 다음, 노름을 지정된 임계값으로 클리핑, 그래디언트의 크기가 제한
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        # 파라미터 업데이트
        optimizer.step()

        # 전체 손실 값 계산
        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [197]:
# 모델 평가(evaluate) 함수
def evaluate(model, iterator, criterion):
    model.eval() # 평가 모드
    epoch_loss = 0

    with torch.no_grad():
        # 전체 평가 데이터를 확인하며
        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg

            # 출력 단어의 마지막 인덱스(<eos>)는 제외
            # 입력을 할 때는 <sos>부터 시작하도록 처리
            output, _ = model(src, trg[:,:-1])

            # output: [배치 크기, trg_len - 1, output_dim]
            # trg: [배치 크기, trg_len]

            output_dim = output.shape[-1]

            output = output.contiguous().view(-1, output_dim)
            # 출력 단어의 인덱스 0(<sos>)은 제외
            trg = trg[:,1:].contiguous().view(-1)

            # output: [배치 크기 * trg_len - 1, output_dim]
            # trg: [배치 크기 * trg len - 1]

            # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
            loss = criterion(output, trg)

            # 전체 손실 값 계산
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

* 학습(training) 및 검증(validation) 진행
    * **학습 횟수(epoch)**: 10

In [198]:
import math
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [199]:
# 메모리에 현재 할당된 메모리량을 바이트 단위로 반환
print(torch.cuda.memory_allocated()/ 1024**2)
print(torch.cuda.memory_cached()/ 1024**2)
# torch.cuda.empty_cache()

603.27734375
31266.0


/opt/conda/lib/python3.10/site-packages/torch/cuda/memory.py:395: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(


In [200]:
# 번역(translation) 함수
# 1. 하나의 sentence가 들어왔을 때
def translate_sentence(sentence, src_field, trg_field, model, device, max_len=50, logging=True): # max_len - change
    model.eval() # 평가 모드
    
    '''
    # 토큰화
    if isinstance(sentence, str):
        nlp = spacy.load('de')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else: # G  문자열이 아닌 경우 이미 토큰화된 문장을 가정하고 각 토큰을 소문자로 변환
        tokens = [token.lower() for token in sentence]
    '''
    # 처음에 <sos> 토큰, 마지막에 <eos> 토큰 붙이기
    src_indexes = [src_field.init_token] + sentence + [src_field.eos_token]
    # if logging:
    #     print(f"전체 소스 토큰: {tokens}")

    # # 모델의 입력으로 넣기 위해 각 단어를 인덱스로 바꿈
    # src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    if logging:
        print(f"소스 문장 인덱스: {src_indexes}")

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)

    # 소스 문장에 따른 마스크 생성
    src_mask = model.make_src_mask(src_tensor)

    # attention_mask = (src_tensor != src_field.pad_token).type(torch.long)

    # 인코더(endocer)에 소스 문장을 넣어 출력 값 구하기
    with torch.no_grad():
        enc_src = model.encoder(src_tensor)
    
    # cls_indices = torch.tensor([0])  # [CLS] 토큰의 인덱스
    # sep_indices = (src == tokenizer.sep_token_id).nonzero()[:, 1]  # [SEP] 토큰의 인덱스
    
    # # 각 배치에 대해 [CLS] 및 [SEP] 토큰의 임베딩을 0으로 설정
    # for batch_idx in range(enc_src.size(0)):
    #     enc_src[batch_idx, 0, :] = 0  # [CLS] 토큰의 임베딩을 0으로 설정
    #     enc_src[batch_idx, -1, :] = 0  
    # print(enc_src)

    # 처음에는 <sos> 토큰 하나만 가지고 있도록 하기
    # ; 실제 출력 문장은 <sos> 토큰부터 출발해서
      # max_len까지 하나씩 반복적으로 모델의 디코더에 넣어서 출력 만듦
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    for i in range(max_len):
        # ?? for문 바깥 위쪽으로 빼면 안되나 -> trg_indexes와 별개로 해야함. 코드 쭉 읽어보기
        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device) 

        # 출력 문장에 따른 마스크 생성
        trg_mask = model.make_trg_mask(trg_tensor)

        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)

        # 매번 디코더에 넣었을 때 마지막 단어가 출력 문장으로써 하나씩 추가됨
        # 출력 문장에서 가장 마지막 단어만 사용
        # G output에서 가장 높은 확률을 가지는 토큰의 인덱스를 추출
          # 2 : 텐서의 차원(axis) 2를 따라 가장 큰 값을 갖는 인덱스를 반환
        pred_token = output.argmax(2)[:,-1].item()
        trg_indexes.append(pred_token) # 출력 문장에 더하기

        # <eos>를 만나는 순간 끝 -> 이때까지 출력된 모든 단어들이 전체 출력 문장이 됨
        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break

    # 각 출력 단어 인덱스를 실제 단어(문자열)로 변환
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]

    # 첫 번째 <sos>는 제외하고 출력 문장 반환
    return trg_tokens[1:], attention

In [201]:
from torchtext.data.metrics import bleu_score

def show_bleu(data, src_field, trg_field, model, device, max_len=50):
    trgs = []
    pred_trgs = []
    index = 0

    for datum in data:
        # G 데이터를 사전 형태로 변환한 후, 'src' 키를 사용하여 해당 데이터의 소스 문장을 추출
        src = vars(datum)['src']
        trg = vars(datum)['trg']

        pred_trg, _ = translate_sentence(src, src_field, trg_field, model, device, max_len, logging=False)

        # 마지막 <eos> 토큰 제거
        pred_trg = pred_trg[:-1]

        pred_trgs.append(pred_trg)
        trgs.append([trg])

        index += 1
        if (index + 1) % 500 == 0:
            print(f"[{index + 1}/{len(data)}]")
            print(f"예측: {pred_trg}")
            print(f"정답: {trg}")

    # G 아래는 디폴트값과 일치 
      # max_n : 사용할 최대 n-gram 크기를 지정
      # eights : 각 n-gram에 대한 가중치를 지정
    bleu = bleu_score(pred_trgs, trgs, max_n=4, weights=[0.25, 0.25, 0.25, 0.25])
    print(f'Total BLEU Score = {bleu*100:.2f}')

    individual_bleu1_score = bleu_score(pred_trgs, trgs, max_n=4, weights=[1, 0, 0, 0])
    individual_bleu2_score = bleu_score(pred_trgs, trgs, max_n=4, weights=[0, 1, 0, 0])
    individual_bleu3_score = bleu_score(pred_trgs, trgs, max_n=4, weights=[0, 0, 1, 0])
    individual_bleu4_score = bleu_score(pred_trgs, trgs, max_n=4, weights=[0, 0, 0, 1])

    print(f'Individual BLEU1 score = {individual_bleu1_score*100:.2f}')
    print(f'Individual BLEU2 score = {individual_bleu2_score*100:.2f}')
    print(f'Individual BLEU3 score = {individual_bleu3_score*100:.2f}')
    print(f'Individual BLEU4 score = {individual_bleu4_score*100:.2f}')

    # G 각 n-gram에 대한 가중치를 모두 동일하게 설정하는 대신에 ㅁ-gram까지의 n-gram을 고려
      # cf) BLEU: 더 긴 n-gram에 대해서는 더 높은 가중치를 부여
    cumulative_bleu1_score = bleu_score(pred_trgs, trgs, max_n=4, weights=[1, 0, 0, 0])
    cumulative_bleu2_score = bleu_score(pred_trgs, trgs, max_n=4, weights=[1/2, 1/2, 0, 0])
    cumulative_bleu3_score = bleu_score(pred_trgs, trgs, max_n=4, weights=[1/3, 1/3, 1/3, 0])
    cumulative_bleu4_score = bleu_score(pred_trgs, trgs, max_n=4, weights=[1/4, 1/4, 1/4, 1/4])

    print(f'Cumulative BLEU1 score = {cumulative_bleu1_score*100:.2f}')
    print(f'Cumulative BLEU2 score = {cumulative_bleu2_score*100:.2f}')
    print(f'Cumulative BLEU3 score = {cumulative_bleu3_score*100:.2f}')
    print(f'Cumulative BLEU4 score = {cumulative_bleu4_score*100:.2f}')

In [ ]:
import time
import math

N_EPOCHS = 40
CLIP = 1
# 최솟값을 찾기 위한 초기값으로 설정
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time() # 시작 시간 기록

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    # valid_loss = evaluate(model, valid_iterator, criterion)
    valid_loss = evaluate(model, test_iterator, criterion)

    end_time = time.time() # 종료 시간 기록
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    # valid_loss가 더 감소한 경우에만 모델 파라미터를 새로운 파일로 기록
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'transformer_german_to_english.pt')

    if epoch % 5 == 4 and epoch != 0:
        show_bleu(test_data, SRC, TRG, model, device)
        
    print(f'{epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s | Train PPL: {math.exp(train_loss):.3f} | Val PPL: {math.exp(valid_loss):.3f}')

01 | Time: 4m 24s | Train PPL: 178.501 | Val PPL: 39.377
02 | Time: 4m 22s | Train PPL: 49.951 | Val PPL: 15.970
03 | Time: 4m 25s | Train PPL: 22.504 | Val PPL: 9.812
04 | Time: 4m 22s | Train PPL: 13.399 | Val PPL: 7.580
[500/3119]
예측: ['besides', 'how', 'did', 'this', 'submarine', 'escape', 'to', 'any', 'public', 'notice']
정답: ['besides', 'how', 'could', 'the', 'assembly', 'of', 'this', 'underwater', 'boat', 'have', 'escaped', 'public', 'notice']
[1000/3119]
예측: ['then', 'you', 'must', 'have', 'a', 'man', 'of', 'business', 'a', 'friend', 'of', 'business', 'you', 'pay', 'a', '<unk>', 'at', 'a', 'rate', 'rate']
정답: ['then', 'you', 'must', 'have', 'some', 'confidant', 'some', 'safe', 'man', 'of', 'business', 'who', 'pays', 'you', 'interest', 'at', 'a', 'fair', 'rate']
[1500/3119]
예측: ['and', 'did', 'you', 'not', 'discover', 'that', 'your', 'clothes', 'were', 'not', 'wet']
정답: ['and', 'was', 'it', 'not', 'discovered', 'that', 'your', 'sheets', 'were', 'unhemmed']
[2000/3119]
예측: ['ah']


In [ ]:
torch.save(model.state_dict(),'transformer_last_trainpt')

In [ ]:
# 학습된 모델 저장
# from google.colab import files

# G 사용자의 로컬 컴퓨터로 다운로드
# files.download('transformer_german_to_english.pt')

#### **모델 최종 테스트(testing) 결과 확인**

In [ ]:
show_bleu(test_data[:2000], SRC, TRG, model, device)

In [ ]:
# show_bleu(train_data[:3000], SRC, TRG, model, device)

In [ ]:
model.load_state_dict(torch.load('transformer_german_to_english.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):.3f}')

#### **나만의 데이터로 모델 사용해보기**

* 테스트 데이터셋을 이용해 모델 테스트 진행

In [ ]:
example_idx = 10

# G vars: 객체의 속성을 사전 형태로 반환하는 파이썬 내장 함수
src = vars(test_data.examples[example_idx])['src']
trg = vars(test_data.examples[example_idx])['trg']

print(f'소스 문장: {src}')
print(f'타겟 문장: {trg}')

# attention : 8개 헤드로 구성된 어센션 스코어들의 집합
translation, attention = translate_sentence(src, SRC, TRG, model, device, logging=True)

print("모델 출력 결과:", " ".join(translation))

* 어텐션 맵(Attention Map) 시각화

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker # G 눈금

# 각 헤드에 대한 어텐션 스코어 값 출력
def display_attention(sentence, translation, attention, n_heads=8, n_rows=4, n_cols=2):

    assert n_rows * n_cols == n_heads

    # 출력할 그림 크기 조절
    fig = plt.figure(figsize=(15, 25))

    for i in range(n_heads):
        ax = fig.add_subplot(n_rows, n_cols, i + 1)

        # 어텐션(Attention) 스코어 확률 값을 이용해 그리기
        # G 배치 차원을 제거
        _attention = attention.squeeze(0)[i].cpu().detach().numpy()

        # G _attention 배열을 매트릭스 형태로 표시
        # G cmap='bone'은 컬러맵을 지정하는 매개변수로, 'bone'은 흑백으로 표시
        cax = ax.matshow(_attention, cmap='bone')

        # 눈금 레이블의 크기를 12로 설정
        ax.tick_params(labelsize=12)
        ax.set_xticklabels([''] + ['<sos>'] + [t.lower() for t in sentence] + ['<eos>'], rotation=45)
        ax.set_yticklabels([''] + translation)

        #  x축의 눈금 간격을 설정
        ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
        ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()
    plt.close()

In [ ]:
example_idx = 10

src = vars(test_data.examples[example_idx])['src']
trg = vars(test_data.examples[example_idx])['trg']

print(f'소스 문장: {src}')
print(f'타겟 문장: {trg}')

translation, attention = translate_sentence(src, SRC, TRG, model, device, logging=True)

print("모델 출력 결과:", " ".join(translation))

In [ ]:
# display_attention(src, translation, attention)
# mother를 출력하기 위해 mutter를 참고했다는 것을 시각적으로 확인 가능

#### <b>BLEU Score 계산</b>

* 학습된 트랜스포머(Transformer) 모델의 BLEU 스코어 계산

In [ ]:
show_bleu(test_data, SRC, TRG, model, device)

In [ ]:
model.load_state_dict(torch.load('transformer_last_trainpt'))